In [1]:
from scipy.stats import multivariate_normal as mvn
from scipy.stats import norm
from scipy.stats import chi2
import numpy as np
import pandas as pd
from scipy import optimize
import math

In [4]:
def d_desc(cf, r, t):
    return cf * math.pow((1 + r), -t) 
def d_cont(cf, r, t):
    return cf * math.exp(-r * t)

In [2]:
# firm valuation using CAPM
# beta_i = cov(ri, rm)/var(rm)

E_CF = 250
rf = 0.05
E_rm = 0.12
beta_i = 1.1


V = E_CF / (1 + rf + beta_i*(E_rm - rf))

print("Value of firm:", round(V, 4))

# forward price
# S0 = E_St * math.exp(-(rf + beta_i * (E_rm - rf)) * T)
# F0 = S0 * math.exp(rf * T)


Value of firm: 221.8279


In [3]:
# market portfolio
w1 = 0.3
w2 = 0.7

E_r1 = 0.1
E_r2 = 0.2

var_r1 = 0.25
var_r2 = 0.45
corr = 0.5
cov_12 = corr * math.sqrt(var_r1 * var_r2)


w = np.array([w1, w2])
E_r = np.array([E_r1, E_r2])
var_r = np.array([[var_r1, cov_12], 
                  [cov_12, var_r2]])

# mean and variance
E_port = np.dot(w, E_r)
var_port = np.dot(np.dot(w, var_r), w)

print("mean of portfolio:", round(E_port, 4))
print("variance of portfolio:", round(var_port, 4))
print()


# beta
beta = np.dot(w, var_r) / var_port
print("beta of asset 1:", round(beta[0], 4))
print("beta of asset 2:", round(beta[1], 4))

mean of portfolio: 0.17
variance of portfolio: 0.3134

beta of asset 1: 0.6138
beta of asset 2: 1.1655


In [ ]:
# beta_i = cov(ri, rp) / var(rp)
# for 2 stock situation:
#      beta1 = (w1*std1**2 + w2*rho*std1*std2)/(std_po**2)
#      beta2 = (w2*std2**2 + w1*rho*std1*std2)/(std_po**2)

In [5]:
# unhedged unlevered firm
rf = 6 / 100

P_up = 50 / 100
P_down = 1 - P_up

CF_up = 480
CF_down = 200

tax_rate = 40 / 100
tax_hurdle = 300

# parameter end


def tax(CF, tax_r, hurdle):
    return max(CF - hurdle, 0) * tax_r

tax_up = tax(CF_up, tax_rate, tax_hurdle)
tax_down = tax(CF_down, tax_rate, tax_hurdle)
_E_CF_1 = P_up * (CF_up - tax_up) + P_down * (CF_down - tax_down)
PV = d_desc(_E_CF_1, rf, 1)

print("Present Value:", round(PV, 4))

Present Value: 286.7925


In [6]:
# hedged unlevered firm
rf = 6 / 100

P_up = 50 / 100
P_down = 1 - P_up

CF_up = 480
CF_down = 200

tax_rate = 40 / 100
tax_hurdle = 300

# parameter end


def tax(CF, tax_r, hurdle):
    return max(CF - hurdle, 0) * tax_r

CF_rn = P_up * CF_up + P_down * CF_down
_E_CF_1 = CF_rn - tax(CF_rn, tax_rate, tax_hurdle)
PV = d_desc(_E_CF_1, rf, 1)

print("Present Value:", round(PV, 4))

Present Value: 305.6604


In [7]:
# unhedged with optimal debt
rf = 0.06

P_up = 50 / 100
P_down = 1 - P_up

CF_up = 480
CF_down = 200

tax_rate = 40 / 100
tax_hurdle = 300

# parameter end


def tax(CF, tax_r, hurdle):
    return max(CF - hurdle, 0) * tax_r

max_safe_debt = d_desc(min(CF_up, CF_down), rf, 1)

tax_up = tax(CF_up - max_safe_debt * rf, tax_rate, tax_hurdle)
tax_down = tax(CF_down - max_safe_debt * rf, tax_rate, tax_hurdle)
_E_CF_1 = P_up * (CF_up - tax_up) + P_down * (CF_down - tax_down)
PV = d_desc(_E_CF_1, rf, 1)

print("Max riskless debt:", round(max_safe_debt, 4))
print("Present Value:", round(PV, 4))

Max riskless debt: 188.6792
Present Value: 288.9284


In [8]:
# hedged with optimal debt
rf = 6 / 100

P_up = 50 / 100
P_down = 1 - P_up

CF_up = 480
CF_down = 200

tax_rate = 40 / 100
tax_hurdle = 300

# parameter end


def tax(CF, tax_r, hurdle):
    return max(CF - hurdle, 0) * tax_r

def mySolver(_sol_dbt):
    CF_rn = P_up * CF_up + P_down * CF_down
    lhs = d_desc(CF_rn - tax(CF_rn - _sol_dbt * rf, tax_rate, tax_hurdle), rf, 1)
    rhs = _sol_dbt  
    return lhs - rhs

result = optimize.fsolve(mySolver, 1)
max_safe_debt = result[0]
PV = mySolver(max_safe_debt) + max_safe_debt

print("Max riskless debt:", round(max_safe_debt, 4))
print("Present Value:", round(PV, 4))

Max riskless debt: 312.7413
Present Value: 312.7413


In [9]:
# unhedged with distinct debt
rf = 0.06

P_up = 50 / 100
P_down = 1 - P_up

CF_up = 480
CF_down = 200

debt = 220
bkupt = 30

tax_rate = 40 / 100
tax_hurdle = 300

# parameter end


def tax(CF, tax_r, hurdle):
    return max(CF - hurdle, 0) * tax_r

def mySolver(_sol_ytm):
    lhs = P_up * d_desc(debt, _sol_ytm, -1) + P_down * (CF_down - bkupt)
    rhs = d_desc(debt, rf, -1) 
    return lhs - rhs

result = optimize.fsolve(mySolver, 1)
ytm = result[0]

_CF_up = CF_up - tax(CF_up - debt * ytm, tax_rate, tax_hurdle)
_CF_down = CF_down - bkupt
_E_CF_1 = P_up * _CF_up + P_down * _CF_down
PV = d_desc(_E_CF_1, rf, 1)

print("YTM:", round(ytm, 6))
print("Present Value:", round(PV, 4))

YTM: 0.347273
Present Value: 287.0566


In [10]:
# unhedged with unknown debtm, considering bankuptcy
rf = 0.06

P_up = 50 / 100
P_down = 1 - P_up

CF_up = 480
CF_down = 200

bkupt = 30

tax_rate = 40 / 100
tax_hurdle = 300

# parameter end


def tax(CF, tax_r, hurdle):
    return max(CF - hurdle, 0) * tax_r

def mySolver(x):
    _debt, _ytm = x.tolist()
    
    lhs1 = P_up * d_desc(_debt, _ytm, -1) + P_down * (CF_down - bkupt)
    rhs1 = d_desc(_debt, rf, -1) 
    lhs2 = d_desc(_debt, _ytm, -1) + tax((CF_up - _debt * _ytm), tax_rate, tax_hurdle)
    rhs2 = CF_up
    return [lhs1 - rhs1,
            lhs2 - rhs2]

result = optimize.fsolve(mySolver, [1, 0.1])
[debt, ytm] = result
tax_up = tax(CF_up - debt * ytm, tax_rate, tax_hurdle)
_E_CF_1 = P_up * (CF_up - tax_up) + P_down * (CF_down - bkupt)
PV = d_desc(_E_CF_1, rf, 1)

print("Debt:", round(debt, 4))
print("YTM:", round(ytm, 6))
print("Present Value:", round(PV, 4))

Debt: 305.0239
YTM: 0.562667
Present Value: 305.0239


In [11]:
# unhedged with distinct debt, 2-depth decision tree
rf = 5 / 100

P_up = 50 / 100
P_down = 1 - P_up

CF_up = 500
CF_down = 200

debt = 250
bkupt = 20

tax_rate = 40 / 100
tax_hurdle = 300

P_gamble_up = 0.005
P_gamble_down = 1 - P_gamble_up
gamble_up = 1000
gamble_down = -20

# parameter end


def tax(CF, tax_r, hurdle):
    return max(CF - hurdle, 0) * tax_r

def mySolver(_sol_ytm):
    lhs = P_up * d_desc(debt, _sol_ytm, -1) + P_down * (P_gamble_down*(CF_down-bkupt+gamble_down) + P_gamble_up*d_desc(debt, _sol_ytm, -1))
    rhs = d_desc(debt, rf, -1) 
    return lhs - rhs

result = optimize.fsolve(mySolver, 1)
ytm = result[0]

_vf = P_up*(CF_up-tax(CF_up-debt*ytm,tax_rate,tax_hurdle)) + \
      P_down*(P_gamble_down*(CF_down-bkupt+gamble_down) + P_gamble_up*(gamble_up+CF_down-tax(gamble_up+CF_down-debt*ytm,tax_rate,tax_hurdle)))
value_firm = d_desc(_vf, rf, 1)

print("YTM:", round(ytm, 6))
print("PV of firm:", round(value_firm, 4))

YTM: 0.45592
PV of firm: 299.6286
